In [127]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Send a request to the website
url = 'https://ceypetco.gov.lk/historical-prices/'
response = requests.get(url)

# Parse the HTML content of the page with BeautifulSoup
soup = BeautifulSoup(response.content, 'html.parser')

# Find the table on the page
table = soup.find('table')

# Extract the rows of the table into a list
rows = table.find_all('tr')

# Initialize lists to store the dates and LAD values
dates = []
lad_values = []

# Loop through each row of the table, skipping the first (header) row
for row in rows[1:]:
    # Extract the date and LAD values from the row
    cells = row.find_all('td')
    date = cells[0].text
    lad = cells[3].text.strip()
    
    # Add the date and LAD values to their respective lists
    dates.append(date)
    lad_values.append(lad)

# Create a dataframe with the dates and LAD values
df = pd.DataFrame({'Date': dates, 'LAD': lad_values})

In [128]:
df.head()

,Date,LAD
0,01.02.2023,405.00
1,03.01.2023,405.00
2,06.12.2022,420.00
3,17.10.2022,430.00
4,12.11.2022,430.00


In [129]:
df['Date'].unique()

array(['01.02.2023', '03.01.2023', '06.12.2022', '17.10.2022',
       '12.11.2022', '17.10.2022(9.00 PM)', '02.10.2022', '21.08.2022',
       '01.08.2022', '17.07.2022', '26.06.2022', '24.05.2022',
       '22.04.2022', '18.04.2022', '22.03.2022', '11.03.2022',
       '27.12.2021', '20.12.2021', '11.06.2021', '10.09.2019',
       '13.08.2019', '11.07.2019', '11.06.2019', '11.05.2019',
       '13.03.2019', '12.02.2019', '11.01.2019', '22.12.2018',
       '01.12.2018', '16.11.2018', '02.11.2018', '11.10.2018',
       '11.09.2018', '04.09.2018', '11.08.2018', '11.07.2018',
       '13.06.2018', '11.05.2018', '28.11.2015', '29.01.2015',
       '22.01.2015', '01.01.2015', '05.12.2014', '17.09.2014',
       '23.02.2013', '14.12.2012', '12.02.2012', '30.10.2011',
       '01.04.2011', '01.09.2010', '29.12.2009', '02.07.2009',
       '30.12.2008', '06.12.2008', '07.11.2008', '25.05.2008',
       '14.01.2008', '29.07.2007', '30.06.2007', '12.05.2007',
       '28.04.2007', '19.04.2007', '05.01.2007

In [130]:
df.at[5, "Date"] = "17.10.2022"

In [131]:
df.head(6)

,Date,LAD
0,01.02.2023,405.00
1,03.01.2023,405.00
2,06.12.2022,420.00
3,17.10.2022,430.00
4,12.11.2022,430.00
5,17.10.2022,415.00


In [132]:
df['Date'] = pd.to_datetime(df['Date'], format='%d.%m.%Y')
df = df.sort_values(by='Date')

# convert the LAD values to numeric
df['LAD'] = pd.to_numeric(df['LAD'], errors='coerce')

In [133]:
# Monthly dataframe that takes the latest LAD value for each month
df_monthly = df.groupby(pd.Grouper(key='Date', freq='MS')).last()

df_monthly.head()

,LAD
Date,
1990-03-01,9.6
1990-04-01,NaN
1990-05-01,NaN
1990-06-01,NaN
1990-07-01,NaN


In [134]:
df_monthly.isnull().sum()

LAD    283
dtype: int64

In [135]:
# Replicating missing values with LAD values from the previous month
df_monthly.fillna(method='ffill', inplace=True)

In [136]:
import plotly.express as px

# Create a line plot of the LAD values over time using Plotly
fig = px.line(df_monthly, x=df_monthly.index, y='LAD', title='LAD Prices over Time (Distributed Monthly)')
fig.show()

In [138]:
# Save the fuel prices to a CSV file
df_monthly.reset_index(drop=False, inplace=True)
df_monthly.to_csv('../dataset/unprocessed/fuel_prices.csv',index=False)